In [6]:
import torch
import pyperclip

path_occupancy = '/Users/nankivel/Desktop/personal_github/capstone/chesscog/models/transfer_learning_models/occupancy_classifier/ResNet.pt'
path_piece_classifier = '/Users/nankivel/Desktop/personal_github/capstone/chesscog/models/transfer_learning_models/piece_classifier/InceptionV3.pt'
# model_occupancy = torch.load(path_occupancy, map_location=torch.device('cpu'))
# pyperclip.copy(str(model_occupancy))

model_classifier = torch.load(
    path_piece_classifier, map_location=torch.device('cpu'))
pyperclip.copy(str(model_classifier))

In [10]:
import torch.nn as nn

# Pruning layers

# Ensure that we're accessing the Inception3 module's 'fc' attribute correctly
num_features = model_classifier.model.fc.in_features

# Now replace the 'fc' layer with a new one tailored to the number of classes you desire (12 in this case)
model_classifier.model.fc = nn.Linear(num_features, 12)


# Fine tuning

# Freeze all layers except the last one
for param in model_classifier.model.parameters():
    param.requires_grad = False

# Unfreeze the last layer
for param in model_classifier.model.fc.parameters():
    param.requires_grad = True

In [22]:
import cv2
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms


class ChessDataset(Dataset):
    def __init__(self, image_paths, fen_strings, transform=None):
        self.image_paths = image_paths
        self.fen_strings = fen_strings
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image = cv2.imread(self.image_paths[index])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            image = self.transform(image)

        fen = self.fen_strings[index]
        # Convert FEN string to your desired format here
        fen_vector = self.fen_to_vector(fen)

        # result = TensorDataset(image, fen_vector)

        return image, fen_vector

    def fen_to_vector(self, fen):
        # Initialize an empty vector for each square
        # 64 squares, 12 possible states per square (no explicit empty square)
        board_vector = np.zeros((64, 12))

        # Define a mapping from pieces to vector indices
        piece_to_index = {
            'p': 0, 'n': 1, 'b': 2, 'r': 3, 'q': 4, 'k': 5,  # Black pieces
            'P': 6, 'N': 7, 'B': 8, 'R': 9, 'Q': 10, 'K': 11  # White pieces
        }

        # Process the FEN string (ignoring castling, en passant, etc.)
        positions = fen.split()[0]
        row = 0
        col = 0

        for char in positions:
            if char.isdigit():
                col += int(char)  # Move to the right for empty squares
            elif char == '/':
                row += 1
                col = 0
            else:
                index = piece_to_index[char]
                board_vector[row * 8 + col, index] = 1
                col += 1

        return board_vector.flatten()  # Flatten to make it a single vector per board


# Define your transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    # Resize to the input size expected by InceptionV3
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])


# Assuming you have lists: image_paths and fen_strings
test_image = '/Users/nankivel/Desktop/personal_github/capstone/data/raw/test1.jpg'
image_paths = [test_image for _ in range(100)]
n = len(image_paths)
starting_fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
fen_strings = [starting_fen for _ in range(n)]

dataset = ChessDataset(image_paths, fen_strings, transform=transform)
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)
# Assuming X_train and y_train are your training features and labels as PyTorch tensors
# train_dataset = TensorDataset(X_train, y_train)

# Creating the DataLoader for batching and shuffling
# train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

learning_rate = .001
num_epochs = 10

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model_classifier.model.parameters(), lr=learning_rate)

# Assuming you have a loader for your training data
for epoch in range(num_epochs):
    for images, labels in train_loader:
        # Forward pass
        outputs = model_classifier(images)
        # Check if the output is a tuple (which occurs with InceptionV3)
        if isinstance(outputs, tuple):
            outputs = outputs[0]  # Extract the main output (logits)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

RuntimeError: 0D or 1D target tensor expected, multi-target not supported